In [107]:
#import the necessary liberies
import os
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO, BytesIO
import datetime as datetime #
import datetime as dt


In [119]:
# read the dataset into pandas dataframe 
df = pd.read_csv(r'C:\Users\ubior\Desktop\zipco_food_orchestraion_with_apache_airflow\clean_data.csv')
df 

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,27.154342,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-02-11 11:00:00,Vanilla Cake,2,11.224568,North,Credit Card,True,Rainy,28.787946,Poor,...,29,In Store,Christopher Joyce,79921 Morrison Summit Suite 437\r\nWest Alyssa...,517.264.6998,rachaelmatthews@example.com,Lauren Hines,robertmaxwell@example.com,Saturday,22.449136
996,2023-02-11 12:00:00,Carrot Cake,3,15.691665,East,Credit Card,False,Cloudy,34.641175,Good,...,32,In Store,Rebecca Hines,"67050 Turner Harbors\r\nMasonfurt, ID 19366",417.298.9399,michaelmorgan@example.com,Roy Garcia,johnsonangela@example.org,Saturday,47.074995
997,2023-02-11 13:00:00,Cheesecake,3,10.793180,West,Cash,True,Sunny,27.984565,Average,...,43,Phone Order,Patricia Turner,"206 Ronald Square Suite 698\r\nJustinmouth, MI...",(655)728-3078,donnarobbins@example.org,Bruce Chung,lori61@example.org,Saturday,32.379540
998,2023-02-11 14:00:00,Cheesecake,5,13.827574,East,Online,True,Rainy,33.579827,Poor,...,52,Online Order,Kathryn Nichols,"7355 Donna Shoals Apt. 887\r\nLake Kevinmouth,...",+1-766-493-7034,mary38@example.org,Leslie Carter,zreed@example.com,Saturday,69.137870


In [120]:
# checak the data structure of the dataset 
df.shape

(1000, 21)

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             1000 non-null   float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        1000 non-null   object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   o

In [122]:
df.drop_duplicates(inplace =True)

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             1000 non-null   float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        1000 non-null   object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   o

In [125]:
# split the customername to firstname and lastname
split_data= df['CustomerName'].str.split(' ',n=1, expand=True)
df['firstname'] =split_data[0]
df['lastname'] =split_data[1].fillna('')


In [126]:
#data cleaing process 
df['CustomerAddress'] = df['CustomerAddress'].str.replace('\r\n', ' ', regex=False)
df['CustomerAddress']


0                  9851 David Green Tonyaburgh, VA 02853
1             24682 Holly Stravenue Mooreville, NH 13901
2      10184 Washington Trace Apt. 679 East Brandista...
3                  87194 Jeff Rue Mitchellbury, CA 50463
4                    850 Julia Groves Hartview, WI 95954
                             ...                        
995    79921 Morrison Summit Suite 437 West Alyssafur...
996             67050 Turner Harbors Masonfurt, ID 19366
997    206 Ronald Square Suite 698 Justinmouth, MI 95340
998    7355 Donna Shoals Apt. 887 Lake Kevinmouth, ME...
999           12000 Kyle Radial West Jasontown, AS 64560
Name: CustomerAddress, Length: 1000, dtype: object

In [127]:
df['Customer_PhoneNumber'] = df['Customer_PhoneNumber'].str.lstrip('(').str.strip(')').str.replace('.','-')
df['Customer_PhoneNumber']=df['Customer_PhoneNumber'].str.replace(')','-').str.replace('x','-')
df['Customer_PhoneNumber']

0      916-427-7276-861
1          769-318-4373
2          702-520-3286
3          622-527-9530
4          759-517-8359
             ...       
995        517-264-6998
996        417-298-9399
997        655-728-3078
998     +1-766-493-7034
999    691-679-2549-867
Name: Customer_PhoneNumber, Length: 1000, dtype: object

In [128]:
# design of the customer table 
customer = df[['firstname','lastname', 'CustomerAddress', 'Customer_PhoneNumber','CustomerEmail','CustomerFeedback']].drop_duplicates()
customer['customer_id'] =range(1,len(df)+1)
customer =customer[['customer_id','firstname','lastname', 'CustomerAddress', 'Customer_PhoneNumber','CustomerEmail','CustomerFeedback']].drop_duplicates()
customer


,customer_id,firstname,lastname,CustomerAddress,Customer_PhoneNumber,CustomerEmail,CustomerFeedback
0,1,William,Adams,"9851 David Green Tonyaburgh, VA 02853",916-427-7276-861,lisa00@example.net,Neutral
1,2,Anthony,Wiggins,"24682 Holly Stravenue Mooreville, NH 13901",769-318-4373,michellefernandez@example.com,Positive
2,3,Ashley,Duke,10184 Washington Trace Apt. 679 East Brandista...,702-520-3286,cooperwilliam@example.com,Unknown
3,4,Brandon,Taylor,"87194 Jeff Rue Mitchellbury, CA 50463",622-527-9530,fsilva@example.net,Positive
4,5,Brittany,Watkins,"850 Julia Groves Hartview, WI 95954",759-517-8359,petersjoseph@example.net,Neutral
...,...,...,...,...,...,...,...
995,996,Christopher,Joyce,79921 Morrison Summit Suite 437 West Alyssafur...,517-264-6998,rachaelmatthews@example.com,Negative
996,997,Rebecca,Hines,"67050 Turner Harbors Masonfurt, ID 19366",417-298-9399,michaelmorgan@example.com,Positive
997,998,Patricia,Turner,"206 Ronald Square Suite 698 Justinmouth, MI 95340",655-728-3078,donnarobbins@example.org,Unknown
998,999,Kathryn,Nichols,"7355 Donna Shoals Apt. 887 Lake Kevinmouth, ME...",+1-766-493-7034,mary38@example.org,Positive


In [129]:
#create the product table 
product = df[['ProductName', 'PaymentType','UnitPrice','StoreLocation','PromotionApplied','OrderType']].drop_duplicates()
product['product_id'] = range(1,len(product)+1)
product=product[['product_id','ProductName', 'PaymentType','UnitPrice','StoreLocation','PromotionApplied','OrderType']]
product

,product_id,ProductName,PaymentType,UnitPrice,StoreLocation,PromotionApplied,OrderType
0,1,Vanilla Cake,Cash,12.532304,South,True,In Store
1,2,Red Velvet Cake,Cash,7.083070,South,False,In Store
2,3,Chocolate Cake,Cash,6.736064,North,True,Phone Order
3,4,Carrot Cake,Cash,7.314823,North,False,Online Order
4,5,Pizza Pepperoni,Credit Card,7.577727,East,True,In Store
...,...,...,...,...,...,...,...
995,996,Vanilla Cake,Credit Card,11.224568,North,True,In Store
996,997,Carrot Cake,Credit Card,15.691665,East,False,In Store
997,998,Cheesecake,Cash,10.793180,West,True,Phone Order
998,999,Cheesecake,Online,13.827574,East,True,Online Order


In [130]:
#create the staff table 
staff = df[['Staff_Name', 'Staff_Email', 'StaffPerformanceRating']].drop_duplicates().reset_index(drop=True)
staff['staff_id'] = staff.index + 1
staff=staff[['staff_id','Staff_Name', 'Staff_Email', 'StaffPerformanceRating']]
staff


,staff_id,Staff_Name,Staff_Email,StaffPerformanceRating
0,1,John Bridges,pdavidson@example.com,Poor
1,2,Sarah Bentley,ajohnson@example.net,Average
2,3,Connie Cervantes,michele29@example.net,Excellent
3,4,Jessica Stewart,xwilson@example.org,Poor
4,5,Cheryl Carpenter,christine96@example.org,Good
...,...,...,...,...
995,996,Lauren Hines,robertmaxwell@example.com,Poor
996,997,Roy Garcia,johnsonangela@example.org,Good
997,998,Bruce Chung,lori61@example.org,Average
998,999,Leslie Carter,zreed@example.com,Poor


In [143]:
#create the date table 
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date']).dt.date
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()
df['Year'] = pd.to_datetime(df['Date']).dt.year
Date =df[['Date','Day','Month','Year']].drop_duplicates().reset_index(drop=True)
Date['date_id']=Date.index+1
Date[['date_id','Date','Day','Month','Year']].head(2)




,date_id,Date,Day,Month,Year
0,1,2023-01-01,Sunday,January,2023
1,2,2023-01-02,Monday,January,2023


In [151]:
#create the transaction table 

transaction = df.merge(customer,on=['firstname','lastname', 'CustomerAddress', 'Customer_PhoneNumber','CustomerEmail','CustomerFeedback'], how ='left')\
    .merge(product, on=['ProductName', 'PaymentType','UnitPrice','StoreLocation','PromotionApplied','OrderType'], how ='left')\
        .merge(staff, on=['Staff_Name', 'Staff_Email', 'StaffPerformanceRating'], how ='left')
transaction = transaction[['Quantity', 'UnitPrice','customer_id','product_id','staff_id','Date','TotalSales','DayOfWeek']] .drop_duplicates().reset_index(drop=True) 
transaction['transaction_id'] = transaction.index+1  
transaction[['transaction_id','customer_id','product_id','staff_id','Quantity','TotalSales', 'UnitPrice','Date','DayOfWeek']] .drop_duplicates()
transaction.head(3)   

,Quantity,UnitPrice,customer_id,product_id,staff_id,Date,TotalSales,DayOfWeek,transaction_id
0,2,12.532304,1,1,1,2023-01-01,25.064608,Sunday,1
1,1,7.083070,2,2,2,2023-01-01,7.083070,Sunday,2
2,5,6.736064,3,3,3,2023-01-01,33.680321,Sunday,3


In [152]:
#create csv file of the table 
transaction.to_csv('transaction.csv', index =False)
customer.to_csv('customer.csv', index =False)
product.to_csv('product.csv', index =False)
staff.to_csv('staff.csv', index =False)
Date.to_csv('Date.csv', index =False)

In [159]:
#%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()



False

In [163]:
#load the data into Azure blob starage
container_string =os.getenv('container_str')
container_name =os.getenv('container_na')

In [ ]:
blob_service_client = BlobServiceClient.from_connection_string(container_string)
container_client = blob_service_client.get_container_client(container_name)
try:
    if not container_client.exists()
    print(f'container {container_name} does not exit')
    container_client.create_container()
    print(f'container{container_name} create successfully')
    